In [1]:

import pandas as pd
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.nn as dglnn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import pickle
import random
import numpy as np
import os
from dgl.nn import GraphConv, GATConv


In [2]:
import pandas as pd

# 讀取CSV檔案
csv_file_path = r'D:\CODE\multi-model knowledge graph multi-graph recommendation system\data\final_data_cleaned.csv'
df = pd.read_csv(csv_file_path)

# 顯示前幾行數據確認讀取是否正確
print(df.head())


   movieId                                              title  \
0    53519                                 Death Proof (2007)   
1    54995                               Planet Terror (2007)   
2    55063                                 My Winnipeg (2007)   
3    55069  4 Months, 3 Weeks and 2 Days (4 luni, 3 saptam...   
4    56102   Endgame: Blueprint for Global Enslavement (2007)   

                                   genres     imdbId   tmdbId     tconst  \
0  Action|Adventure|Crime|Horror|Thriller  tt1028528   1991.0  tt1028528   
1                    Action|Horror|Sci-Fi  tt1077258   1992.0  tt1077258   
2                     Documentary|Fantasy  tt1093842  13241.0  tt1093842   
3                                   Drama  tt1032846   2009.0  tt1032846   
4                             Documentary  tt1135489  18312.0  tt1135489   

                                           crew_info  
0  actor: nm0000621, actress: nm1057928, actress:...  
1  actress: nm0000535, actor: nm0135585, a

In [3]:
def parse_crew_info(crew_info):
    crew_dict = {
        'actor': [],
        'actress': [],
        'director': [],
        'writer': [],
        'producer': [],
        'composer': [],
        'editor': [],
        'self': []
    }
    for item in crew_info.split(','):
        role, person_id = item.strip().split(': ')
        crew_dict[role].append(person_id)
    return crew_dict

# 應用函數來解析每一行的crew_info
df['crew_parsed'] = df['crew_info'].apply(parse_crew_info)
print(df[['movieId', 'crew_parsed']].head())


   movieId                                        crew_parsed
0    53519  {'actor': ['nm0000621', 'nm0000233'], 'actress...
1    54995  {'actor': ['nm0135585', 'nm0000982', 'nm000119...
2    55063  {'actor': ['nm0991671', 'nm0624369', 'nm241163...
3    55069  {'actor': ['nm0412096', 'nm2308578', 'nm230470...
4    56102  {'actor': ['nm1093953', 'nm1093953', 'nm109395...


In [5]:


# Load user ratings
ratings_path = r'D:\\CODE\\multi-model knowledge graph multi-graph recommendation system\\data\\cleanuser_rating.csv'
ratings_data = pd.read_csv(ratings_path)

# Load the graph
graph_path = r'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\0.消融實驗\updated_hetero_graph03_with_V_T_A_features.pkl'
with open(graph_path, 'rb') as f:
    graph = pickle.load(f)


In [6]:
# Example to print the graph and its features
# Verify the graph structure and features
print(graph)
print("Node types:", graph.ntypes)
print("Edge types:", graph.etypes)

# Check for features in specific node types
for ntype in ['movietext', 'movieimage', 'movieaudio']:
    if 'features' in graph.nodes[ntype].data:
        print(f"Features for {ntype} nodes:", graph.nodes[ntype].data.keys())
        print(f"Shape of features for {ntype} nodes:", graph.nodes[ntype].data['features'].shape)
    else:
        print(f"No features found for {ntype} nodes.")


Graph(num_nodes={'actor': 4566, 'actress': 3701, 'composer': 2011, 'director': 4010, 'editor': 2489, 'movie': 5996, 'movieaudio': 5996, 'movieimage': 5996, 'movietext': 5996, 'producer': 3952, 'self': 1049, 'user': 12171, 'writer': 3177},
      num_edges={('movie', 'has_actor', 'actor'): 9809, ('movie', 'has_actress', 'actress'): 8327, ('movie', 'has_audio', 'movieaudio'): 5996, ('movie', 'has_composer', 'composer'): 5923, ('movie', 'has_director', 'director'): 10163, ('movie', 'has_editor', 'editor'): 4089, ('movie', 'has_image', 'movieimage'): 5996, ('movie', 'has_producer', 'producer'): 9891, ('movie', 'has_self', 'self'): 2138, ('movie', 'has_text', 'movietext'): 5996, ('movie', 'has_writer', 'writer'): 7036, ('movie', 'similar', 'movie'): 4852, ('user', 'rates', 'movie'): 549919, ('user', 'similar', 'user'): 39895602},
      metagraph=[('movie', 'actor', 'has_actor'), ('movie', 'actress', 'has_actress'), ('movie', 'movieaudio', 'has_audio'), ('movie', 'composer', 'has_composer'), 

In [7]:

print("Graph loaded successfully.")
print("Node types:", graph.ntypes)
print("Edge types:", graph.etypes)
# Check for features in all node types
for ntype in graph.ntypes:
    print(f"\nNode type: {ntype}")
    print(f"Features for {ntype} nodes:", graph.nodes[ntype].data.keys())
    for key in graph.nodes[ntype].data.keys():
        print(f"Shape of '{key}' for {ntype} nodes:", graph.nodes[ntype].data[key].shape)
# Check for specific relationships
relationships = [
    ('movie', 'has_actor', 'actor'),
    ('movie', 'has_actress', 'actress'),
    ('movie', 'has_director', 'director'),
    ('movie', 'has_producer', 'producer'),
    ('movie', 'has_writer', 'writer')
]

for rel in relationships:
    u, v = graph.edges(etype=rel)
    print(f"Relationship {rel}:")
    print(f"  Number of edges: {len(u)}")
    print(f"  Source nodes (sample): {u[:5]}")
    print(f"  Destination nodes (sample): {v[:5]}")


Graph loaded successfully.
Node types: ['actor', 'actress', 'composer', 'director', 'editor', 'movie', 'movieaudio', 'movieimage', 'movietext', 'producer', 'self', 'user', 'writer']
Edge types: ['has_actor', 'has_actress', 'has_audio', 'has_composer', 'has_director', 'has_editor', 'has_image', 'has_producer', 'has_self', 'has_text', 'has_writer', 'similar', 'rates', 'similar']

Node type: actor
Features for actor nodes: dict_keys([])

Node type: actress
Features for actress nodes: dict_keys([])

Node type: composer
Features for composer nodes: dict_keys([])

Node type: director
Features for director nodes: dict_keys([])

Node type: editor
Features for editor nodes: dict_keys([])

Node type: movie
Features for movie nodes: dict_keys(['movie_id'])
Shape of 'movie_id' for movie nodes: torch.Size([5996])

Node type: movieaudio
Features for movieaudio nodes: dict_keys(['movie_id', 'features', 'audio_features'])
Shape of 'movie_id' for movieaudio nodes: torch.Size([5996])
Shape of 'features'

In [8]:
import dgl
import torch



# Function to check features for a specific node type
def check_features(graph, node_type):
    if node_type in graph.ntypes:
        if 'features' in graph.nodes[node_type].data:
            features = graph.nodes[node_type].data['features']
            print(f"Features are available in '{node_type}' nodes.")
            print(f"Shape of the features: {features.shape}")
            print(f"Average of features: {torch.mean(features)}")
            print(f"Non-zero elements in features: {torch.count_nonzero(features)}")
            if node_type == 'movietext' and features.shape[1] != 128:  # Adjusted expected dimension
                print(f"Incorrect dimension of features for {node_type}: {features.shape[1]}")
            elif node_type == 'movieimage' and features.shape[1] != 2048:  # Expected dimension for image
                print(f"Incorrect dimension of features for {node_type}: {features.shape[1]}")
            elif node_type == 'movieaudio' and features.shape[1] != 128:  # Expected dimension for audio
                print(f"Incorrect dimension of features for {node_type}: {features.shape[1]}")
        else:
            print(f"No features data found in '{node_type}' nodes.")
    else:
        print(f"No '{node_type}' node type found in the graph.")



# Check features for each modality
check_features(graph, 'movietext')
check_features(graph, 'movieimage')
check_features(graph, 'movieaudio')


Features are available in 'movietext' nodes.
Shape of the features: torch.Size([5996, 384])
Average of features: 0.0
Non-zero elements in features: 0
Incorrect dimension of features for movietext: 384
Features are available in 'movieimage' nodes.
Shape of the features: torch.Size([5996, 2048])
Average of features: 0.25277090072631836
Non-zero elements in features: 8816537
Features are available in 'movieaudio' nodes.
Shape of the features: torch.Size([5996, 128])
Average of features: 0.0
Non-zero elements in features: 0


In [17]:
# Assuming userID and movieID need to be encoded
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

ratings_data['userId'] = user_encoder.fit_transform(ratings_data['userId'])
ratings_data['movieId'] = movie_encoder.fit_transform(ratings_data['movieId'])
print(ratings_data.head())
from sklearn.model_selection import train_test_split

# Split into training and temp (validation + test)
train_data, temp_data = train_test_split(ratings_data, test_size=0.3, random_state=42)

# Split temp into validation and test
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

import torch
from torch.utils.data import DataLoader, Dataset

class RatingsDataset(Dataset):
    def __init__(self, dataframe):
        self.users = torch.tensor(dataframe['userId'].values, dtype=torch.long)
        self.items = torch.tensor(dataframe['movieId'].values, dtype=torch.long)
        self.ratings = torch.tensor(dataframe['rating'].values, dtype=torch.float)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

# Create dataset objects
train_dataset = RatingsDataset(train_data)
validation_dataset = RatingsDataset(validation_data)
test_dataset = RatingsDataset(test_data)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



   userId  movieId  rating   timestamp
0       0        1     4.0  1251170120
1       0        9     3.5  1230788571
2       0       35     2.0  1230788649
3       0       88     2.0  1251170520
4       0      118     4.0  1294796033


In [16]:
print(ratings_data.head())
print(f"Train data size: {len(train_data)}")
print(f"Validation data size: {len(validation_data)}")
print(f"Test data size: {len(test_data)}")
# 檢查第一個 batch 的形狀
for users, items, ratings in train_loader:
    print(f"Users batch shape: {users.shape}")
    print(f"Items batch shape: {items.shape}")
    print(f"Ratings batch shape: {ratings.shape}")
    break


   userId  movieId  rating   timestamp
0       0        1     4.0  1251170120
1       0        9     3.5  1230788571
2       0       35     2.0  1230788649
3       0       88     2.0  1251170520
4       0      118     4.0  1294796033
Train data size: 384943
Validation data size: 82488
Test data size: 82488
Users batch shape: torch.Size([64])
Items batch shape: torch.Size([64])
Ratings batch shape: torch.Size([64])


In [ ]:
def get_negative_samples(user_items, item_pool, num_negatives):
    negative_samples = {}
    item_pool_list = list(item_pool)  # Convert set to list for sampling
    for user, positive_items in user_items.items():
        negative_samples[user] = []
        while len(negative_samples[user]) < num_negatives:
            negative_item = np.random.choice(item_pool_list)
            if negative_item not in positive_items:
                negative_samples[user].append(negative_item)
    return negative_samples

# Assuming `ratings_data` from previous steps
user_items = ratings_data.groupby('userId')['movieId'].apply(set).to_dict()
all_items = set(ratings_data['movieId'].unique())

# Generate negative samples for each user
num_negatives = 5  # Adjust based on how many negatives per positive
negative_samples = get_negative_samples(user_items, all_items, num_negatives)


In [ ]:
# First, let's print the number of positive and negative samples for a few users
for user in list(user_items.keys())[:5]:  # Check the first 5 users
    print(f"User {user}: {len(user_items[user])} positive samples, {len(negative_samples[user])} negative samples")

# Constructing train_samples from positive and negative samples
train_samples = []
for user, positives in user_items.items():
    for positive in positives:
        for negative in negative_samples[user]:  # Ensuring there's a loop over negatives
            train_samples.append((user, positive, negative))

# Now, let's inspect the first few training samples
print("\nExample training samples:")
for sample in train_samples[:5]:
    print(f"User {sample[0]}, Positive Item {sample[1]}, Negative Item {sample[2]}")

# 
# Finally, let's check the distribution of the number of items per user
positive_counts = [len(items) for items in user_items.values()]
negative_counts = [len(items) for items in negative_samples.values()]

print("\nStatistics of positive samples per user:")
print(f"Mean: {np.mean(positive_counts)}, Median: {np.median(positive_counts)}, Min: {np.min(positive_counts)}, Max: {np.max(positive_counts)}")
print("\nStatistics of negative samples per user:")
print(f"Mean: {np.mean(negative_counts)}, Median: {np.median(negative_counts)}, Min: {np.min(negative_counts)}, Max: {np.max(negative_counts)}")


User 0: 16 positive samples, 80 negative samples
User 1: 41 positive samples, 205 negative samples
User 2: 24 positive samples, 120 negative samples
User 3: 55 positive samples, 275 negative samples
User 4: 92 positive samples, 460 negative samples

Example training samples:
User 0, Positive Item 159, Negative Item 3933
User 0, Positive Item 159, Negative Item 3318
User 0, Positive Item 159, Negative Item 4800
User 0, Positive Item 159, Negative Item 4022
User 0, Positive Item 159, Negative Item 2553

Statistics of positive samples per user:
Mean: 45.18272943883001, Median: 23.0, Min: 10, Max: 1067

Statistics of negative samples per user:
Mean: 225.91364719415003, Median: 115.0, Min: 50, Max: 5335


In [ ]:
# import pandas as pd

# # Convert training samples list to DataFrame
# train_samples_df = pd.DataFrame(train_samples, columns=['User', 'PositiveItem', 'NegativeItem'])

# # Save DataFrame to CSV
# train_samples_df.to_csv('training_samples.csv', index=False)

# print("Training samples saved to 'training_samples.csv'.")


In [ ]:
# import numpy as np
# import pandas as pd
# import csv  # 導入 csv 模組

# def get_negative_samples(user_items, item_pool, num_negatives):
#     negative_samples = {}
#     for user, positive_items in user_items.items():
#         possible_negatives = list(item_pool - positive_items)
#         negative_samples[user] = np.random.choice(possible_negatives, num_negatives, replace=False).tolist()
#     return negative_samples

# # 假設有前面步驟的 `ratings_data`
# user_items = ratings_data.groupby('userId')['movieId'].apply(set).to_dict()
# all_items = set(ratings_data['movieId'].unique())

# # 為每個用戶生成負樣本
# num_negatives = 5  # 根據每個正樣本需要的負樣本數量進行調整
# negative_samples = get_negative_samples(user_items, all_items, num_negatives)

# # 函數直接批量將樣本寫入 CSV
# def write_samples_to_csv(user_items, negative_samples, filename='training_samples.csv'):
#     with open(filename, 'w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['User', 'PositiveItem', 'NegativeItem'])
#         for user, positives in user_items.items():
#             for positive in positives:
#                 for negative in negative_samples[user]:
#                     writer.writerow([user, positive, negative])

# # 直接將訓練樣本寫入 CSV 文件
# write_samples_to_csv(user_items, negative_samples)

# print("訓練樣本已保存到 'training_samples.csv'。")


In [10]:
# Load training samples from CSV
train_samples_df = pd.read_csv(r'D:\CODE\multi-model knowledge graph multi-graph recommendation system\data\training_samples.csv')
print("Training samples loaded from 'training_samples.csv'.")

# Display the first few rows to verify the data
print(train_samples_df.head(10))


Training samples loaded from 'training_samples.csv'.
   User  PositiveItem  NegativeItem
0     0           159          3436
1     0           159          4175
2     0           159          3818
3     0           159          2636
4     0           159          2977
5     0             1          3436
6     0             1          4175
7     0             1          3818
8     0             1          2636
9     0             1          2977


In [11]:
import torch
from torch.utils.data import DataLoader, Dataset

class BPRDataset(Dataset):
    def __init__(self, dataframe):
        self.users = torch.tensor(dataframe['User'].values, dtype=torch.long)
        self.pos_items = torch.tensor(dataframe['PositiveItem'].values, dtype=torch.long)
        self.neg_items = torch.tensor(dataframe['NegativeItem'].values, dtype=torch.long)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.pos_items[idx], self.neg_items[idx]

# Create dataset object
train_dataset = BPRDataset(train_samples_df)

# Create dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Check the first batch to verify
for batch in train_loader:
    users, pos_items, neg_items = batch
    print("Users batch:", users)
    print("Positive items batch:", pos_items)
    print("Negative items batch:", neg_items)
    break  # Just check the first batch


Users batch: tensor([ 9705,   805,  8431,  3248,    60,    44,  4882,  3025,  6604,  2331,
         7035,  9015,  2679,  5673,   320,  1447,  8053,  6995,  5673, 11707,
        10587,  6120,  7885,  6501,  4177,  4478,  7081,  3020,  4562,  3930,
        11969,  5333,  4104,  4452, 11242, 10811,  1782,  2339,  1098,  2049,
         4377,  3184,  5385,  9775,  3535,  2087,   958,  1913,  2691, 10698,
          554,   953,  7144, 10969, 10846,  7513,  6011,    41,  9500, 11618,
        11628,  6522,  1420, 10099])
Positive items batch: tensor([ 227, 2114, 3636,  369,  903, 3715, 1755,   93,  421, 2011,  793,  279,
         549,  793,  928, 1926, 1233,  201, 2186,   90, 1712, 1672, 1781, 1827,
        2675, 1430,    0,  282,  195,    8,  121, 2784, 3450, 1373, 3752, 1554,
         561,  660, 1774,  277, 2545, 3203,  686, 1040, 1737,  208, 4428,  421,
           9,  601,  328, 1182, 5171,    3,  252, 3459,  304,  210,  901, 1627,
        1859,  208,  372, 2520])
Negative items batch: tenso

In [12]:
import pandas as pd
import torch



# Determine the number of unique users and items
num_users = train_samples_df['User'].nunique()
num_items = train_samples_df[['PositiveItem', 'NegativeItem']].nunique().sum()

# Create tensors for user, positive item, and negative item indices
user_indices = torch.tensor(train_samples_df['User'].values, dtype=torch.long)
item_indices_pos = torch.tensor(train_samples_df['PositiveItem'].values, dtype=torch.long)
item_indices_neg = torch.tensor(train_samples_df['NegativeItem'].values, dtype=torch.long)

print("User indices:", user_indices)
print("Positive item indices:", item_indices_pos)
print("Negative item indices:", item_indices_neg)


User indices: tensor([    0,     0,     0,  ..., 12170, 12170, 12170])
Positive item indices: tensor([159, 159, 159,  ..., 507, 507, 507])
Negative item indices: tensor([3436, 4175, 3818,  ..., 2561, 4251, 3461])


In [13]:
num_users = train_samples_df['User'].nunique()
num_items = train_samples_df[['PositiveItem', 'NegativeItem']].nunique().sum()  # Count unique positive and negative items
print(num_items)
print(num_users)

11991
12171


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn.pytorch import GATConv

class MKGMR(nn.Module):
    def __init__(self, num_features, num_users, num_items, hidden_dim, out_dim, num_heads, num_layers, dropout_rate=0.5):
        super(MKGMR, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.common_dim = num_features  # Target common dimension for all modalities
        
        # Embeddings for users and items
        self.user_embeddings = nn.Embedding(num_users, num_features)
        self.item_embeddings = nn.Embedding(num_items, num_features)
        
        # Transformation layers for each modality
        self.text_transform = nn.Linear(384, self.common_dim)
        self.image_transform = nn.Linear(2048, self.common_dim)
        self.audio_transform = nn.Linear(128, self.common_dim)
        
        # GAT layers
        self.gat_layers = nn.ModuleList()
        self.dropout_layers = nn.ModuleList()
        for _ in range(num_layers):
            self.gat_layers.append(GATConv(self.common_dim, hidden_dim, num_heads))
            self.dropout_layers.append(nn.Dropout(dropout_rate))
            self.common_dim = hidden_dim * num_heads
        self.gat_layers.append(GATConv(self.common_dim, out_dim, 1))  # The final layer with 1 head
        self.dropout_layers.append(nn.Dropout(dropout_rate))
        
        # Final projection layers
        self.user_transform = nn.Linear(out_dim, out_dim)
        self.item_transform = nn.Linear(out_dim, out_dim)

    def forward(self, g, user_indices, item_indices_pos, item_indices_neg, text_features, image_features, audio_features):
        # Embed users and items
        user_emb = self.user_embeddings(user_indices)
        item_emb_pos = self.item_embeddings(item_indices_pos)
        item_emb_neg = self.item_embeddings(item_indices_neg)
        
        # Transform and aggregate multimodal features
        text_emb = self.text_transform(text_features)
        image_emb = self.image_transform(image_features)
        audio_emb = self.audio_transform(audio_features)
        
        movie_emb = (text_emb + image_emb + audio_emb) / 3  # Simple average for aggregation
        
        # Apply GAT layers with dropout and symmetric normalization
        x = torch.cat([user_emb, item_emb_pos, item_emb_neg, movie_emb], dim=0)
        g.ndata['h'] = x
        g = dgl.add_self_loop(g)  # Adding self-loops for normalization
        degs = g.in_degrees().float().clamp(min=1)
        norm = torch.pow(degs, -0.5)
        norm = norm.to(x.device).unsqueeze(1)

        for gat_layer, dropout_layer in zip(self.gat_layers, self.dropout_layers):
            x = gat_layer(g, g.ndata['h'])
            x = x * norm
            x = F.relu(x)
            x = dropout_layer(x)
            g.ndata['h'] = x

        # Separate the embeddings
        user_emb, item_emb_pos, item_emb_neg, movie_emb = torch.split(x, [len(user_indices), len(item_indices_pos), len(item_indices_neg), len(movie_emb)], dim=0)

        # Transform embeddings for final scoring
        user_emb = self.user_transform(user_emb)
        item_emb_pos = self.item_transform(item_emb_pos)
        item_emb_neg = self.item_transform(item_emb_neg)

        # Compute scores
        pos_scores = torch.sum(user_emb * item_emb_pos, dim=1)
        neg_scores = torch.sum(user_emb * item_emb_neg, dim=1)

        return pos_scores, neg_scores

    def bpr_loss(self, pos_scores, neg_scores):
        # Bayesian Personalized Ranking loss
        loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10))
        return loss

    def regularization_loss(self):
        # L2 regularization for embeddings
        reg_loss = (self.user_embeddings.weight.norm(2) + self.item_embeddings.weight.norm(2)) * 0.01
        return reg_loss

    def training_step(self, g, user_indices, item_indices_pos, item_indices_neg, text_features, image_features, audio_features):
        pos_scores, neg_scores = self.forward(g, user_indices, item_indices_pos, item_indices_neg, text_features, image_features, audio_features)
        bpr_loss = self.bpr_loss(pos_scores, neg_scores)
        reg_loss = self.regularization_loss()
        loss = bpr_loss + reg_loss
        return loss


In [15]:

# Example of how to use the class
if __name__ == "__main__":
    # Dummy data for demonstration purposes
    num_features = 32
    hidden_dim = 64
    out_dim = 32
    num_heads = 2
    num_layers = 3
    dropout_rate = 0.5

    model = MKGMR(num_features, num_users, num_items, hidden_dim, out_dim, num_heads, num_layers, dropout_rate)

    # Dummy graph for demonstration purposes
    # In practice, this should be the actual graph from your dataset
    g = dgl.graph(([0, 1, 2], [1, 2, 3]))

    # Dummy indices for demonstration purposes
    user_indices = torch.tensor([0, 1, 2])
    item_indices_pos = torch.tensor([0, 1, 2])
    item_indices_neg = torch.tensor([3, 4, 5])

    # Perform a single training step
    loss = model.training_step(g, user_indices, item_indices_pos, item_indices_neg)
    print("Training loss:", loss.item())

TypeError: training_step() missing 3 required positional arguments: 'text_features', 'image_features', and 'audio_features'